<a href="https://colab.research.google.com/github/liamssi/vscolab/blob/master/vscolab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

># 1.WORKSPACE CONFIGURATION

In [ ]:
"""
This will configure some envirement variabs to be used later in further steps to
as parameters to set the workspace for you
"""
import os


### GIT CONFIG
#(REQUIRED/OPTIONAL)required/optional depending on your setup (private or public git)
os.environ['GIT_USER']='' #your github user: FIRST_NAME LAST_NAME   
os.environ['GITHUB_EMAIL']='' #your github email

#(OPTIONAL)this repo will be used as the default workspace to open in vs code 
os.environ['GITHUB_REPO'] = '' #IT'S IMPORTANT TO REMOVE THE 'https://' PART OF THE URL

#(OPTIONAL) you only need to set these if you are using a private repo 
os.environ['PRIVAT_REPO'] ="true" #ANY STRING OTHER THAN 'FALSE' WILL BE TAKEN AS TRUE (CASE INSENSITIVE)
os.environ['GITHUB_USERNAME']=''  
os.environ['GITHUB_PASS'] =""



# (OPTIONAL)NGROK CONFIG 
#IF YOU NEED TO EXPOSE OTHER ORTS THIS ID REQUIRED AS NGROK ONLY PERMIT ONE TUNNEL FOR NON AUTHENTICATED USERS
#os.environ['NGROK_TOKEN']=''

#VSCODE CONFIG 
#(REQUIRED)local port to launch VS CODE SERVER on 
os.environ['VSCODE_PORT'] = '8389' #YOU PORBABLY DON'T NEED TO CHANGE THIS

#(OPTIONAL) other local ports to expose(EXAMPLE: your web dev server port)
#you may declare many multip ports seperated by spaces ('OTHER_PORTS') but that requires a paid ngrok example: os.environ['OTHER_PORTS'] = '3000 3001 3002'
#os.environ['OTHER_PORTS'] = '3000' #CHANGE THIS DEPENDING ON THE PORT YOUR WEB SERVER USES

># 2. INSTALL REQUIRED TOOLS 
---
> * install code editor VS Code - [Code Server](https://github.com/cdr/code-server) 
> * install port forwading tool [Ngrok](https://ngrok.com/) and its Python wrapper [PyNgrok](https://pyngrok.readthedocs.io/)
> * install the latest version of [Node.Js](https://nodejs.org/)


In [ ]:
%%shell
#install VS Code server 
echo 
echo
echo "--------------------------"
echo "- Installing code server -"
echo "--------------------------"
echo
echo
curl -fsSL https://code-server.dev/install.sh | sh

#download and install ngrok tool
echo
echo
echo "--------------------------"
echo "-   Installing Ngrock    -"
echo "--------------------------"
echo
echo
wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip  #Download
unzip -qq -n  ngrok-stable-linux-amd64.zip #Unzip 
#Pyngrok will installl the binaries for us 
pip install pyngrok #install ngrok python Python wrapper (this is better way to install because it automaticaly addes ngrok to the PATH)
#echo 'ngrok installed ...'

#install a newer version of node.Js
echo
echo
echo "--------------------------"
echo "-   Installing node.Js   -"
echo "--------------------------"
echo
echo

curl -sL https://deb.nodesource.com/setup_14.x | sudo -E bash - #Add Node.js PPA to sources files 
apt install nodejs -y #Install node.JS 

># 3.WORKSPACE CONFIGURATION 
---
> > * set basic configuration for git client (See [Docs](https://git-scm.com/docs/git-config) )
> * clone your gihub repo if exists
> * generate ngrok config to (See [Docs](https://ngrok.com/docs))

In [ ]:
#git configuration 
%%shell
git config --global --replace-all user.name  "${GIT_USER_NAME}"
git config --global  --replace-all user.email $GITHUB_EMAIL
#CLONE REPO 
PRIVAT_REPO=${PRIVAT_REPO,,}
  if [[ $GITHUB_REPO  &&  $PRIVAT_REPO && $PRIVAT_REPO != "false" ]] ; then
       REPO="https://${GITHUB_USERNAME}:${GITHUB_PASS}@${GITHUB_REPO}"
  elif [ $GITHUB_REPO ]; then 
      REPO="https://${GITHUB_REPO}"
  fi
#echo ${REPO}
git clone $REPO

In [ ]:
#generate a config file for ngrok
%%shell
mkdir /root/.ngrok2/ #create ngrok config dir if not exist

if [ $NGROK_TOKEN ]; then
       echo "authtoken: ${NGROK_TOKEN}" > /root/.ngrok2/ngrok.yml
fi
echo "tunnels:
  vscode:
    addr: ${VSCODE_PORT}
    proto: http
    inspect: false
    region: eu">> /root/.ngrok2/ngrok.yml

for PORT  in ${OTHER_PORTS[@]}; do
  echo "  APP_${PORT}:
    addr: ${PORT}
    proto: http
    inspect: false
    region: eu">> /root/.ngrok2/ngrok.yml
done

cat /root/.ngrok2/ngrok.yml

># 4.START THE WORKSPACE
---
> > * start VS Code Server (See [Docs](https://github.com/cdr/code-server/blob/master/doc/guide.md#3-expose-code-server) )
> * start ngrok tunneling to expose local envirement to the internet (See [Docs](https://ngrok.com/docs))

In [ ]:
#--------------------------#
#     launch workspace     #   
#--------------------------#
#start code server (on port VSCODE_PORT)
import time
GITHUB_REPO=os.environ['GITHUB_REPO']
REPONAME=GITHUB_REPO[GITHUB_REPO.rfind("/")+1:GITHUB_REPO.rfind(".git")] #extract the repo name from github url

cmd=f"code-server ./{REPONAME} --bind-addr 127.0.0.1:{os.environ['VSCODE_PORT']} &"

get_ipython().system_raw(cmd)
get_ipython().system_raw("ngrok start --all &")
#sleep for while and wait for the background services to start
time.sleep(10)

In [ ]:

#--------------------------#
#     helpper function     #   
#--------------------------#
import psutil
import requests


#this is a hellper function that will be used to kill background services later if needed
def get_tunnels():
  try:
    r=requests.get('http://127.0.0.1:4040/api/tunnels')
    tunnels=r.json()['tunnels']
    print("[!] YOUR ACTIVE TUNNELS ARE :")
  except:
    print("Can't access ngrok local api")
    print("Are you sure you ngrok is running?")
    return ''
  for tunnel in tunnels:
    if 'https' in tunnel['public_url']:
      print(f"{ tunnel['name'].ljust(10) }  : {tunnel['public_url']}")
  return tunnels

#kill proces
def kill_process(procname):
  proc_count=0
  for proc in psutil.process_iter():
    # check whether the process name matches
    if proc.name() == procname:
      proc.kill()
      #print(proc.name())
      proc_count+=1
  print(f"killed {proc_count} process..")


># 4.PROVIDE WORKSPACE INFOS 
---
> * here you will find all the infos you need to start using your workspace
> * these infos include :
>   * public URL for your VS Code workspace
>   * public URL for your web server
>   * your VS Code password

> #### NOTE: To end your workspace you simply run :
> ```
> kill_process("ngrok")
>```

In [ ]:
#result
tuns=get_tunnels()
print("*****************************")
print("[!] YOUR VS CODE PASSWORD IS   :")

!cat ~/.config/code-server/config.yaml|grep password:

In [ ]:
#uncomment and run bellow line to stop exposing your workspace to the internet
#kill_process("ngrok")